<a href="https://colab.research.google.com/github/chaiminwoo0223/Deep-Learning/blob/main/15%20-%20Char_RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Download

In [1]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

--2023-05-08 12:00:04--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2023-05-08 12:00:04 (172 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [2]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)


# Import

In [3]:
import torch
import torch.nn as nn
import unidecode
import string
import random
import re
import time, math

# Hyperparameter

In [4]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

# Data
## 1.Prepare Characters

In [5]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print("num_chars =", n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars = 100


## 2.Get Text Data

In [6]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print("file_len =", file_len)

file_len = 1115394


# Functions For Text Processing
## 1.Random Chunk

In [7]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

hither bear your treasure and your goods.
For my part, I'll resign unto your grace
The seal I keep: and so betide to me
As well I tender you and all of yours!
Come, I'll conduct you to the sanctuary.




## 2.Character To Tensor

In [8]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor
print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


## 3.Chunk Into Input & Label

In [9]:
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

# LSTM

## 1.Model

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(1,-1))
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.decoder(out.view(batch_size,-1))
        return out, hidden, cell
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)      

In [11]:
inp = char_tensor("A")
print(inp)
hidden, cell = model.init_hidden()
print(hidden.size())
out, hidden, cell = model(inp, hidden, cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


## 2.Loss & Optimizer

In [12]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## 3.Test Function

In [13]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden, cell = model.init_hidden()
    x = inp
    print(start_str, end="")

    for i in range(200):
        output, hidden, cell = model(x, hidden, cell)
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        print(predicted_char, end="")
        x = char_tensor(predicted_char)

# Train

In [14]:
for i in range(num_epochs):
    inp, label = random_training_set() # 랜덤한 텍스트 덩어리를 샘플링하고, 이를 텐서로 변환한다.
    hidden, cell = model.init_hidden()
    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()

    for j in range(chunk_len-1):
        x = inp[j]
        y_= label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden, cell = model(x, hidden, cell)
        loss += loss_func(y, y_)
    
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print("\n", loss/chunk_len, "\n")
        test()
        print("\n")


 tensor([4.5762], grad_fn=<DivBackward0>) 

QWhkOgL(FiU^P^oc:]z2GXo#R'O^E6<vtKjcPyU!axDL/o#znV}x!9nL{e7.ib[`MP6F5|GIerm,19b|m&CwvXR;>A|OT*>^wpw9 
@}6uNo
T#r[QVvbW?7XJP&y5hKY_7)V+	,3ycy~+BMh.01nRN,/6Zaf


 tensor([2.7900], grad_fn=<DivBackward0>) 

blono bis Iy onceer the in  bhes tlk gSt sov fy mamth erere mo, the spas art to mherer he bithe swhinodrwrl tacha is weri teedan gorlt lat lher moe che toufird  ale sit arthe or m It
swoutd ir thhas Sa


 tensor([2.3730], grad_fn=<DivBackward0>) 

bmee tore lis wninmen0ing, blthanend the yowi ndom Rpin wins worerlende that fears dot loth loflul
That for phis ofYlenet asge ing arteind be lifrtisd the me, whot my mome-heat whirnd cof sealat ise sa


 tensor([2.2643], grad_fn=<DivBackward0>) 

bma'd hass our the of then zouy famut the beart sawh me pliry soJ ther.
Ring tiis an you, this:
Thou hare thert me the thurting:
Whale minde,

OIN:
INor oullll id Lome the fertim win lo soit may of the


 tensor([2.2994], grad_fn=<DivBackward0>) 

bed las